In [1]:
import gymnasium as gym
from gymnasium.spaces import Discrete, Box
import numpy as np
import os
import random

import ray
from ray import air, tune
from ray.rllib.env.env_context import EnvContext
from ray.rllib.utils.framework import try_import_tf, try_import_torch
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.tune.logger import pretty_print
from ray.tune.registry import get_trainable_cls

from custom_env import CustomEnvironment
from config import run_config

tf1, tf, tfv = try_import_tf()
torch, nn = try_import_torch()


if __name__ == "__main__":
    
    class Args:
        def __init__(self):
            self.run = "PPO"
            self.framework = "torch"
            self.as_test = False
            self.stop_iters = 50
            self.stop_timesteps = 100000
            self.stop_reward = 0.1
            self.local_mode = False
    
    args = Args()
    print(f"Running with following options: {args.__dict__}")

    ray.init(local_mode=args.local_mode)
    env = CustomEnvironment(run_config["env"])

    config = (
        get_trainable_cls(args.run)
        .get_default_config()
        .environment(CustomEnvironment, env_config=run_config["env"])
        .framework(args.framework)
        .rollouts(num_rollout_workers=1)
        # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
        .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
    )


    config["env"] = CustomEnvironment
    config["env_config"] = run_config["env"]

    print(config)
    config.multi_agent(
        policies= {
            "prey": (None, env.observation_space, env.action_space, {}),
            "predator": (None, env.observation_space, env.action_space, {}),
        },
        policy_mapping_fn = lambda x: "prey" if x <=env.num_prey else "predator",
    )

    stop = {
        "training_iteration": args.stop_iters,
        "timesteps_total": args.stop_timesteps,
        "episode_reward_mean": args.stop_reward,
    }
    
    # automated run with Tune and grid search and TensorBoard
    print("Training automatically with Ray Tune")
    tuner = tune.Tuner(
        args.run,
        param_space=config.to_dict(),
        run_config=air.RunConfig(stop=stop),
    )
    results = tuner.fit()

    if args.as_test:
        print("Checking if learning goals were achieved")
        check_learning_achieved(results, args.stop_reward)

    ray.shutdown()

2023-08-07 13:54:22,103	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(pid=53220) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=53220) 2023-08-07 13:54:28,829	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config=